In [1]:
# 1.1)
import os
import pickle
import datetime
from collections import defaultdict

In [2]:
# 1.2)
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy as sp
import scipy.linalg as linalg
#import sklearn as skl
import pandas as pd
#import dill
import json

In [3]:
# 1.3
import torch
import torch.optim as optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda, Compose
#from torchviz import make_dot

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.version.cuda)

cuda
11.8


In [ ]:
# 2.1)
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
train_set_orig = datasets.FashionMNIST('MNIST_data/', download = True, train = True,  transform = transform)
valid_set_orig = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transform)


print(len(train_set_orig))  # 60000
print(len(valid_set_orig))        # 10000''''''

In [ ]:
# 3.1)
# Creamos una subclase de la clase Dataset que nos sirva para generar lotes de ejemplos que puedan usarse para entrenar un autoencoder
class CustomDataset(Dataset):
    def __init__(self,dataset):
        self.dataset=dataset
    # Redefinimos el método .__len__()
    def __len__(self):
        return len(self.dataset)
    # Redefinimos el método .__getitem__()
    def __getitem__(self,i):
        image,label=self.dataset[i]
        input  = image
        output = image #torch.flatten(image) # retornamos la imagen como salida
        return input,output

In [ ]:
# 3.2)
# Convertimos FashionMNIST Dataset a CustomDataset
train_set = CustomDataset(train_set_orig)
valid_set = CustomDataset(valid_set_orig)

In [ ]:
class Autoencoder0(nn.Module):
    def __init__(self, n, p=0.2):
        super(Autoencoder0, self).__init__()

         # Encoder convolucional
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),  # -> 14x14
            nn.Dropout(p),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),  # -> 7x7
            nn.Dropout(p),
            nn.Flatten(), # ->32*7*7 = 1568 
            nn.Linear(32*7*7, n), #-> n
        )

        # Decoder convolucional
        self.decoder = nn.Sequential(
            nn.Linear(n, 32*7*7),
            nn.Unflatten(dim=1, unflattened_size=(32,7,7)),
            nn.ConvTranspose2d(32, 16, 2, stride=2),  # -> 16x14x14
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 2, stride=2),   # -> 1x28x28
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)

        return x


mi_red = Autoencoder0(128,p=0.2)